In [4]:
import numpy as np


In [9]:
# dataset generation
# points
A = np.matrix([[10.0,10.0,10.0],
               [20.0,10.0,10.0],
               [20.0,10.0,15.0]])

B = np.matrix([[18.8106,17.6222,12.8169],
               [28.6581,19.3591,12.8173],
               [28.9554, 17.6748, 17.5159]])


# transformations
T_src = np.matrix([[0.9848, 0.1737,0.0000,-11.5865],
                   [-0.1632,0.9254,0.3420, -7.621],
                   [0.0594,-0.3369,0.9400,2.7752],
                   [0.0000, 0.0000,0.0000,1.0000]])
T_dst = np.matrix([[0.9848, 0.1737,0.0000,-11.5859],
                   [-0.1632,0.9254,0.3420, -7.621],
                   [0.0594,-0.3369,0.9400,2.7755],
                   [0.0000, 0.0000,0.0000,1.0000]])

print(A.shape)
print(T_src.shape)

scaling = False

(3, 3)
(4, 4)


In [12]:
def rigid_transform_3D(A, B, scale):

    assert len(A) == len(B)
    N = A.shape[0]

    Am = np.mean(A, axis=0)
    Bm = np.mean(B, axis=0)

    Ac = A - np.tile(Am, (N, 1))
    Bc = B - np.tile(Bm, (N, 1))

    H = np.transpose(Bc) * Ac # NOTE: Kabsch; H = P^T \cdot Q
    if scale: H /= N

    """
    Based on rotatoin formula, optimal rotation R is

    R = sqrt(H^T \cdot H) \cdot H^-1

    since directly solving this is complicated and hard to handle edge cases (e.g., singular H), use SVD
    """

    U, S, Vt = np.linalg.svd(H) # NOTE: Kabsch; H = USV^T
    
    R = Vt.T * U.T # NOTE: Kabsch; R = V \cdot U^T

    # special reflection case
    if np.linalg.det(R) < 0:
        print("[DEBUG] Reflection detected")
        Vt[2, :] *= -1
        R = Vt.T * U.T
    
    if scale:
        Avar = np.var(A, axis=0).sum()
        c = 1 / (1 / Avar * np.sum(S)) # scale singular value
        t = -R * (Bc.T * c) + Ac.T
        
    else:
        c = 1
        t = -R * Bc.T + Ac.T

    return c, R, t

s, R, t = rigid_transform_3D(A, B, False)